In [30]:
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings


warnings.filterwarnings("ignore")
diamonds = sns.load_dataset("diamonds")
diamonds.head()



,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [31]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


In [32]:
import sklearn;
from sklearn.model_selection import train_test_split

"""
diamonds = diamonds.drop('cut', axis=1)
diamonds = diamonds.drop('color', axis=1)
diamonds = diamonds.drop('clarity', axis=1)
diamonds = diamonds.drop('depth', axis=1)
diamonds = diamonds.drop('table', axis=1)
"""

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

X.dtypes

carat    float64
x        float64
y        float64
z        float64
dtype: object

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE 

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

import xgboost as xgb

# Instantiation 
xgb_r = xgb.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 100, seed = 123,enable_categorical=True) 

# Fitting the model 
xgb_r.fit(X_train, y_train) 
  
# Predict the model 
pred = xgb_r.predict(X_test) 
  
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, pred)) 
print("RMSE : % f" %(rmse)) 


RMSE :  1369.499087


In [34]:
#Using Validation

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE 

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

import xgboost as xgb

# Instantiation 
xgb_r = xgb.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 1000, seed = 123,enable_categorical=True,early_stopping_rounds=50) 

# Fitting the model, with VALIDATION with addtnal param eval_set
xgb_r.fit(X_train, y_train,eval_set=[(X_train, y_train),(X_test,y_test)]) 


# Predict the model 
pred = xgb_r.predict(X_test) 
  
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, pred)) 
print("RMSE : % f" %(rmse)) 
print("best iteration is:")
print(xgb_r.best_iteration)


[0]	validation_0-rmse:2974.37884	validation_1-rmse:2919.88741
[1]	validation_0-rmse:2301.28283	validation_1-rmse:2264.04021
[2]	validation_0-rmse:1882.81515	validation_1-rmse:1858.56099
[3]	validation_0-rmse:1635.55328	validation_1-rmse:1621.57283
[4]	validation_0-rmse:1496.90414	validation_1-rmse:1490.53132
[5]	validation_0-rmse:1421.74964	validation_1-rmse:1421.54046
[6]	validation_0-rmse:1380.82795	validation_1-rmse:1387.44017
[7]	validation_0-rmse:1358.53626	validation_1-rmse:1368.89039
[8]	validation_0-rmse:1345.24810	validation_1-rmse:1360.15919
[9]	validation_0-rmse:1336.96567	validation_1-rmse:1355.51371
[10]	validation_0-rmse:1331.91300	validation_1-rmse:1353.83749
[11]	validation_0-rmse:1326.49686	validation_1-rmse:1353.00541
[12]	validation_0-rmse:1323.57877	validation_1-rmse:1352.71699
[13]	validation_0-rmse:1319.76825	validation_1-rmse:1353.09488
[14]	validation_0-rmse:1316.18832	validation_1-rmse:1353.17149
[15]	validation_0-rmse:1313.65032	validation_1-rmse:1352.48380
[1

In [ ]:
results = xgb_r.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["rmse"], label="Training loss")
plt.plot(results["validation_1"]["rmse"], label="Validation loss")
plt.axvline(21, color="gray", label="Optimal tree number")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()